In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import math
import numpy.random as nrand
import numpy as np

In [2]:
stocks_prices = pd.read_csv('../data/DEFINITIVE/prices_from2008_csv.csv', parse_dates =["Date"], index_col="Date")
stocks_mc = pd.read_csv('../data/DEFINITIVE/MC_from2008_csv.csv', parse_dates =["Date"], index_col="Date")
stocks_pb = pd.read_csv('../data/DEFINITIVE/PTB_from2008_csv.csv', parse_dates =["Date"], index_col="Date")
stocks_s_c = pd.read_csv('../data/DEFINITIVE/categories_csv.csv', index_col="sector_country")
returns = stocks_prices.pct_change().dropna()
returns.head()
# returns.iloc[0:60]
# print(returns.head())
# var = returns.iloc[0:60].var()
# std = returns.iloc[0:60].std()
# cov_matrix = returns.iloc[0:60].cov()
# print(var)
# print(std)
# cov_matrix
# # print(type(cov_matrix))
# corr_matrix = returns.iloc[0:60].corr()
# e_r = stocks_prices.iloc[0:60].resample('Y').last().pct_change().mean()
# sd = stocks_prices.iloc[0:60].pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(12))

,AT0000606306,AT0000652011,AT0000730007,AT0000743059,AT0000937503,BE0003470755,BE0003565737,BE0003739530,BE0003764785,BE0003797140,...,SE0000667891,SE0000667925,SE0000695876,SE0001662230,SE0005190238,SE0007100581,SE0007100599,SE0011166610,SE0012455673,SE0013256682
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-31,-0.183880,-0.253608,-0.203619,-0.133887,-0.171284,-0.122362,-0.116528,0.047228,-0.018507,-0.117332,...,-0.184205,-0.076850,-0.072536,-0.161022,-0.004594,-0.154478,-0.144787,-0.067505,-0.297265,-0.144031
2008-02-29,0.010053,0.063536,0.131475,-0.002083,0.020010,-0.008811,-0.019414,-0.034631,0.032086,0.025142,...,0.172743,-0.107659,0.002595,0.062480,-0.145357,0.027653,-0.010921,0.087657,0.254511,-0.024028
2008-03-31,0.011124,0.066234,-0.069076,-0.126096,0.052632,-0.029189,-0.014399,-0.298295,-0.030499,-0.027921,...,-0.024428,-0.037473,0.072596,0.053832,0.023241,-0.038850,-0.006256,0.044358,-0.111978,-0.066052
2008-04-30,0.205559,0.161998,0.099223,0.157430,0.119318,0.168275,0.060872,0.263804,-0.003192,0.056023,...,-0.100364,0.129343,0.098954,-0.131823,0.198826,-0.126255,-0.030981,-0.041582,0.038749,0.135150
2008-05-30,-0.074448,0.028721,0.137886,0.092260,0.087716,-0.024041,-0.089571,-0.068141,0.073639,0.049620,...,0.116511,0.018673,0.025919,-0.003117,-0.037107,0.112279,-0.004670,0.098514,0.038296,0.001202


# Process to follow
1.  I calculate the volatility of the returns.

2.  I classify stocks within dimensions and categories.

3.    For each category, I calculate the sum of (stock volatility/ number of stocks in the category). And then I use this number as Pij.

For example, in dimension Sectors and category Energy, there are 11 stocks. Then I calculate ∑(stock volatility / 11) = Pij

In [3]:
#calculate vol stocks in a certain period.
volatility =  returns.rolling(window=60).std() * np.sqrt(12)
stock_vol = volatility.dropna()

In [4]:
stocks_s_c = pd.read_csv('../data/DEFINITIVE/categories_csv.csv', index_col="sector_country")

##categories
#this is a list containing different sectors and countries
#I want to divide the list in different sectors
#I want to divide the list in different countries

#stocks = stocks_s_c.columns
stocks_s_c = stocks_s_c.transpose()
#stocks_s_c[stocks_s_c["Country "]== "Austria"]

# Create dictionary of stocks by country
countries = stocks_s_c["Country "].unique()
stocks_by_country = {}
for country in countries:
    stocks_by_country[country] = stocks_s_c[stocks_s_c["Country "]== country].index.values.tolist()

# Create dictionary of stocks by sector
sectors = stocks_s_c["Sector name "].unique()
stocks_by_sector = {}
for sector in sectors:
    stocks_by_sector[sector] = stocks_s_c[stocks_s_c["Sector name "]== sector].index.values.tolist()

# dates
dates =  pd.DatetimeIndex(stocks_mc.index)
years = dates.year.unique()
months = dates.month.unique()

# Create dictionary of stocks by market-cap quintile
mean_by_month = stocks_mc
df_mc_quint = {}
for year in years:
    temp_year = mean_by_month[mean_by_month.index.year == year]
    for month in months:

        temp_month = temp_year[temp_year.index.month == month]
        if(len(temp_month)>0):
            df_mc_quint[str(year) + "-" + str(month)] = pd.qcut(temp_month.squeeze(), 5, labels=range(5))

df_mc_quint = pd.DataFrame.from_dict(df_mc_quint)

# Create dictionary of stocks by price-book quintile
pb_by_month = stocks_pb
df_pb_quint = {}
for year in years:
    temp_year = pb_by_month [pb_by_month.index.year == year]
    for month in months:
        temp_month = temp_year[temp_year.index.month == month]
        if(len(temp_month)>0):
            df_pb_quint[str(year) + "-" + str(month)] = pd.qcut(temp_month.squeeze(), 5, labels=range(5))


df_pb_quint = pd.DataFrame.from_dict(df_pb_quint)
print(df_mc_quint.head())
print(df_pb_quint.head())

             2008-1 2008-2 2008-3 2008-4 2008-5 2008-6 2008-7 2008-8 2008-9  \
AT0000606306      3      3      3      3      3      3      3      3      2   
AT0000652011      3      3      3      3      3      3      3      3      3   
AT0000730007      0      0      0      0      0      0      0      0      0   
AT0000743059      3      3      3      3      3      3      3      3      3   
AT0000937503      2      2      2      2      2      2      2      2      1   

             2008-10  ... 2019-7 2019-8 2019-9 2019-10 2019-11 2019-12 2020-1  \
AT0000606306       1  ...      1      1      1       1       1       1      1   
AT0000652011       2  ...      2      2      2       2       2       2      2   
AT0000730007       0  ...      0      0      0       0       0       0      0   
AT0000743059       3  ...      2      2      2       2       2       2      2   
AT0000937503       1  ...      0      0      0       0       0       0      0   

             2020-2 2020-3 2020-4  
AT

In [5]:
import time
dates  = pd.DatetimeIndex(stock_vol.index)
years = dates.year.unique()
volatility_by_month = {}

#volatility per stock per period

for year in years:
    temp_year = stock_vol [stock_vol.index.year == year]
    for month in months:
        temp_month = temp_year[temp_year.index.month == month]
        if(len(temp_month)>0):
            volatility_by_month[str(year) + "-" + str(month)] = temp_month.reset_index(drop=True).loc[0]

volatility_by_month_df  = pd.DataFrame.from_dict(volatility_by_month)
volatility_by_month_df.head()

,2012-12,2013-1,2013-2,2013-3,2013-4,2013-5,2013-6,2013-7,2013-8,2013-9,...,2018-3,2018-4,2018-5,2018-6,2018-7,2018-8,2018-9,2018-10,2018-11,2018-12
AT0000606306,0.567918,0.562317,0.565091,0.566310,0.558428,0.557563,0.557232,0.557324,0.560089,0.540188,...,0.464300,0.468089,0.468404,0.462617,0.463561,0.466027,0.464474,0.462550,0.463449,0.468142
AT0000652011,0.595760,0.583969,0.583523,0.585792,0.582956,0.583249,0.582532,0.589599,0.590027,0.585381,...,0.285372,0.283174,0.288910,0.275375,0.260291,0.261860,0.261502,0.257356,0.257940,0.269155
AT0000730007,0.330976,0.315282,0.314144,0.312355,0.312274,0.316841,0.315861,0.314936,0.313103,0.281427,...,0.232449,0.231284,0.221668,0.221472,0.223005,0.223720,0.223268,0.226408,0.228413,0.229602
AT0000743059,0.348363,0.347087,0.349909,0.345147,0.339568,0.337039,0.336290,0.333132,0.333757,0.298055,...,0.240007,0.240877,0.242033,0.241630,0.240659,0.241637,0.242235,0.241423,0.245351,0.253608
AT0000937503,0.494520,0.488425,0.488776,0.489417,0.486447,0.486332,0.486854,0.480042,0.479337,0.440729,...,0.221754,0.221611,0.219914,0.229481,0.230901,0.229911,0.226652,0.244822,0.245462,0.249571


In [6]:
#main loop 



#generate random porfolios
stocks_names = stocks_mc.columns.values.tolist()
num_portfolios = 1
counter = 0
start = time.time()
porfolios_entropy_sector = {}
porfolios_entropy_country = {}
porfolios_entropy_mc = {}
porfolios_entropy_pb = {}
stock_all_weights={}


for portfolio in range(num_portfolios):
    start_portfolio = time.time()
    print("Portfolio {} ...".format(portfolio))
    num_stocks = len(stocks_s_c)
    
    #random weights-monte carlo simulation
    weights = np.random.random(num_stocks)
    weights = weights/np.sum(weights)
    stock_weights ={}
    for i, stock in enumerate(stocks_names):
        stock_weights[stock] = weights[i]
        
    stock_all_weights[portfolio]=stock_weights
    
    
    #entropy sectors dimension
    sectors = stocks_by_sector.keys()
    dimension = [sectors]
    dimension_sector = {}
    for year in years:
        for month in months:
            pij = 0
            for sector in stocks_by_sector:
                stocks = stocks_by_sector[sector]
                for stock in stocks:
                    key = str(year) + "-" + str(month)
                    if(key in volatility_by_month_df.columns):
                        temp = volatility_by_month_df[key]
                        volatility = ((temp[temp.index == stock].iloc[0]) * stock_weights[stock])/len(stocks)
                        pij += volatility
            pij = pij * np.log(pij)
            dimension_sector[key] = pij
    porfolios_entropy_sector[portfolio] = dimension_sector
    
#     print(dimension_sector)
    
    stop_sector = time.time()
    print("Total time sector = " +str(float(stop_sector-start))+ " seconds")
    
    #entropy country dimension
    dimension_country = {}
    for year in years:
        for month in months:
            pij = 0
            for country in stocks_by_country:
                stocks = stocks_by_country[country]
                for stock in stocks:
                    for weight in weights:
                        key = str(year) + "-" + str(month)
                        if(key in volatility_by_month_df.columns):
                            temp = volatility_by_month[key]
                            volatility = (temp[temp.index == stock].iloc[0]* weight)/len(stocks) # * weight[stock]
                            pij += volatility
            pij = pij * np.log(pij)
            dimension_country[key] = pij
    porfolios_entropy_country[portfolio] = dimension_country
#     porfolios_entropy[portfolio][1] = dimension_country

    stop_country = time.time()
    print("Total time country= " +str(float(stop_country-stop_sector))+ " seconds")
    
    #entropy market-cap dimension
    dimension_mc = {}
    for year in years:
        for month in months:
            pij = 0
            for quintil in range(5):
                key = str(year) + "-" + str(month)
                stocks = df_mc_quint[key][df_mc_quint[key]==quintil].index
                for stock in stocks:
                    for weight in weights:
                        temp = volatility_by_month[key]
                        volatility = (temp[temp.index == stock].iloc[0]* weight)/len(stocks) # * weight[stock]
                        pij += volatility
            pij = pij * np.log(pij)
            dimension_mc[key] = pij
    porfolios_entropy_mc[portfolio] = dimension_country
#         porfolios_entropy[portfolio][2] = dimension_mc
        
    stop_mc = time.time()
    print("Total time mc = " +str(float(stop_mc-stop_country))+ " seconds")
        
    #entropy price-book dimension
    dimension_pb = {}
    for year in years:
        for month in months:
            pij = 0
            for quintil in range(5):
                key = str(year) + "-" + str(month)
                stocks = df_pb_quint[key][df_pb_quint[key]==quintil].index
                for stock in stocks:
                    for weight in weights:
                        temp = volatility_by_month[key]
                        volatility = (temp[temp.index == stock].iloc[0]* weight)/len(stocks) # * weight[stock]
                        pij += volatility
            pij = pij * np.log(pij)
            dimension_pb[key] = pij
    porfolios_entropy_pb[portfolio] = dimension_country
#         porfolios_entropy[portfolio][3] = dimension_pb

    stop_pb = time.time()
    print("Total time mc = " +str(float(stop_pb-stop_mc))+ " seconds")

print("Portfolio {} : DONE in {} seconds".format(portfolio, str(float(time.time()-start_portfolio))))


print("Total time = " +str(float(time.time()-start))+ " seconds")

# print(counter)
# counter+=1

    
# print(porfolios_entropy_country)
# print(porfolios_entropy_sector)    
# print(porfolios_entropy_mc)
# print(porfolios_entropy_pb)

# end = time.time()
# print(end - start) 

Portfolio 0 ...
Total time sector = 17.420167922973633 seconds


<ipython-input-6-744cfb121824>:47: RuntimeWarning: divide by zero encountered in log
  pij = pij * np.log(pij)
<ipython-input-6-744cfb121824>:47: RuntimeWarning: invalid value encountered in double_scalars
  pij = pij * np.log(pij)
<ipython-input-6-744cfb121824>:70: RuntimeWarning: divide by zero encountered in log
  pij = pij * np.log(pij)
<ipython-input-6-744cfb121824>:70: RuntimeWarning: invalid value encountered in double_scalars
  pij = pij * np.log(pij)


KeyboardInterrupt: 

In [ ]:
#First, this cell calculates the joint portfolio entrropy for each period and each weight. 
#Secondly, for each period, it identifies the weights that maximize- the entropy

portfolio_entropy = {}
years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
best_per_year = []
final = pd.DataFrame(index=years)
for portfolio in range(num_portfolios):
    portfolio_results = []
    for year in years:
#         print(year)
#         print(porfolios_entropy_sector[portfolio][year])
    
        sum_total = 0
        sum_total+= porfolios_entropy_sector[portfolio][year]#+ porfolios_entropy_country[portfolio][year]
        sum_total+= porfolios_entropy_mc[portfolio][year]+porfolios_entropy_pb[portfolio][year]
        sum_total = sum_total * -1
        portfolio_entropy[year] = sum_total
#     print(portfolio_entropy.values())
    final[portfolio]=portfolio_entropy.values()


# print(final)
dest=final.idxmax(axis=1)

final=final.max(axis=1)

final
final_weights={}
year=2012
for position in dest:
#     print(position)
    final_weights[year]=stock_all_weights[position]
    year+=1
# print(type(final_weights))



In [ ]:
#In each period, for the MDOP porfolio (the weigts that maximize the porfolio entropy), it calculates sharpe ratio and sortino ratio

i=0
j=60 
returns_all = []
volatility_all = []
sharpe_ratio = []
sortino_list_1 = []
sortino_list_2 = []
weights_best = np.array(final_weights)
# print(weights_best)
# print(final_weights)

for weights in final_weights.values():
#     print(weights.values())
#     print(type(avg_returns))

    avg_returns = returns.iloc[i+60:j+12].mean().to_frame()
#     print(avg_returns)
    weights = pd.DataFrame.from_dict(weights.values())
    returns_max = returns.iloc[i+60:j+12].mean(axis=1)

    weighted_ret = avg_returns.values * weights.values
    weighted_ret = weighted_ret.tolist()
    weighted_ret = [val for sublist in weighted_ret for val in sublist]

    returns_p = (sum(weighted_ret))*12
#     print(returns_p)
    returns_all.append(returns_p)
   
    
    returns_sortino = returns.iloc[i:j]
    weights = pd.DataFrame(weights)
    weights_list = weights.values.tolist()
    weights_list = [val for sublist in weights_list for val in sublist]


    weighted_sortino = returns_sortino *  weights_list
#     print(weighted_sortino)
    returns_p_sortino = weighted_sortino.sum(axis=1)
    
    returns_p_sortino = pd.DataFrame(returns_p_sortino)
    rf_sortino = riskfree.iloc[i:j] / 100 

    
#     print(len(returns_p_sortino.values))
#     print(len(rf_sortino.values))
#     break
    excess_returns_sortino = returns_p_sortino.values - rf_sortino.values

        
    cov_matrix = returns.iloc[i:j].cov()
    var = np.dot(weights.T,np.dot(cov_matrix,weights))
    sd = np.sqrt(var)
    vol_p = sd*np.sqrt(12)
    volatility_all.append(vol_p)
    
    rf = float(riskfree.iloc[j]) / 100
    s_r = (returns_p - rf) / vol_p
    s_r = np.round(s_r,3)
    sharpe_ratio.append(s_r)
    
    

    lpm_1 = (np.maximum(0,  rf - np.asarray(excess_returns_sortino)) ** 2).mean(axis=0)
    lpm_2 = (np.maximum(0,  0 -np.asarray(excess_returns_sortino)) ** 2).mean(axis=0)

    sort1 = ( returns_p - rf) / np.sqrt(lpm_1)
    sort1 = np.round(sort1,3).tolist()
    sortino_list_1.append(sort1)
        
    sort2 = ( returns_p - rf) / np.sqrt(lpm_2)
    sort2 = np.round(sort2,3).tolist()
    sortino_list_2.append(sort2)
        
    
    i+=12
    j+=12
    
print(sharpe_ratio)
print(sortino_list_1)
print(sortino_list_2)
print(returns_all)
print(volatility_all)
    
        